In [1]:
import time
import random
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from numpy import genfromtxt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
# (base) wc -l Ecoli1936_DNAKmerk20Fold1.tsv
# 35951658 Ecoli1936_DNAKmerk20Fold1.tsv
# (base) wc -l Ecoli1936_DNAKmerk31Fold1.tsv
# 44309081 Ecoli1936_DNAKmerk31Fold1.tsv

In [5]:
### Make fold for data
# pa_matrix = pd.read_csv('/data/hoan/amromics/prediction/output/pantaEcoli1936align_v9/gene_presence_absence.Rtab', sep='\t', index_col=0).T
# metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Ecoli1936metafiles/metadata_final.csv")
# sample_list = list(pa_matrix.index)
# sample_list_faa = [sample + '.faa' for sample in sample_list] #, 'SAMEA2204229.contig.fna'

# for idx in range(800, 1200):
# # for idx in range(1200, len(sample_list_faa)):
#     data_dir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/'+sample_list_faa[idx][:-3]
#     # cmd = 'cp -r '+data_dir+'* /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold1' # 0-400
#     cmd = 'cp -r '+data_dir+'fna /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold3' # 400-800
#     # print(cmd)
#     os.system(cmd)

In [2]:
# # for idx in range(800, 1200):
# for idx in range(1200, len(sample_list_faa)):
#     data_dir = '/data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/'+sample_list_faa[idx][:-3]
#     # cmd = 'cp -r '+data_dir+'* /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold1' # 0-400
#     cmd = 'cp -r '+data_dir+'fna /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold4' # 400-800
#     # print(cmd)
#     os.system(cmd)

# First run kounta

In [3]:
# conda activate amromics
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold1.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold1/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold2.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold2/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold3.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold3/*.fna
# kounta --kmer 31 --out /data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold4.tsv /data/hoan/amromics/prediction/data/Ecoli1936/prokkaMore/Fold4/*.fna

In [4]:
metadata_panta = pd.read_csv("/data/hoan/amromics/prediction/data/Ecoli1936metafiles/metadata_final.csv")

# Process DNA data

## Feature selection phase I: select features with high variance

In [5]:
aafold = '1' # '1', '2', '3', '4'

In [7]:
# kmer_matrix_VT1_fi = pd.read_csv('/data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold'+aafold+'.tsv', index_col = 0, sep='\t')

In [ ]:
import timeit

start = timeit.default_timer()
full_data = []
full_name = []
import csv 
count = 0;
with open('/data/hoan/amromics/prediction/data/Ecoli1936_DNAKmerk31Fold'+aafold+'.tsv', 'r') as csvfile: 
    students_reader = csv.reader(csvfile, delimiter='\t') 
    for row in students_reader: 
        data_row = np.array(row[1:])
        if count > 0:
            data_row = data_row.astype(np.int)
            if (np.var(data_row)) > 0.05:
                full_data.append(data_row)
                full_name.append(row[0])
        count += 1
        # if count > 100:
        #     break;

stop = timeit.default_timer()
print('Time: ', stop - start)

In [ ]:
full_data = np.array(full_data)

In [ ]:
feature = np.array(full_name)

In [ ]:
# should use VT5
np.save('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_DNAmat_VT1.npy', full_data) # save numpy array
np.save('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_DNAmat_VT1_features.npy', feature) # save numpy array

In [ ]:
stop

## Feature selection Phase II: select features which correlate with labels

In [8]:
aafold = '1' # '1', '2', '3', '4'

In [6]:
kmer_matrix_VT1_fi = np.load('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_DNAmat_VT1.npy')

In [15]:
selected_features_fi = np.load('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_DNAmat_VT1_features.npy')

In [9]:
kmer_matrix_VT1_fi = kmer_matrix_VT1_fi.T

In [16]:
kmer_matrix_VT1_fi.shape, selected_features_fi.shape

((400, 21073201), (21073201,))

In [17]:
# df1 = pd.DataFrame(data = kmer_matrix_VT1_f1, columns = selected_features_f1)
# df2 = pd.DataFrame(data = kmer_matrix_VT1_f2, columns = selected_features_f2)

In [18]:
# df_full = pd.concat([df0,df1,df2], axis=0, ignore_index=True) # fail: it's too big

In [19]:
from pangraph.utils import binary_label
from sklearn.feature_selection import mutual_info_classif, chi2

In [20]:
if aafold == '1':
    start_idx = 0; end_idx = 400
elif aafold == '2':
    start_idx = 400; end_idx = 800
elif aafold == '3':
    start_idx = 800; end_idx = 1200
else:
    start_idx = 1200; end_idx = metadata_panta.shape[0]

In [ ]:
mutual_mat = []
for idx in range(2, 14):
    # y_class = metadata_panta.iloc[:600,idx].values
    y_class = metadata_panta.iloc[start_idx:end_idx, idx].values
    print(metadata_panta.columns[idx])
    y, nonenan_index = binary_label(y_class) # v6
    pa_matrix_new = kmer_matrix_VT1_fi[nonenan_index, ]
    y_new = y[nonenan_index].astype(int)
    if len(y_new) > 10:
        scores, pvalue = chi2(pa_matrix_new, y_new)
        mutual_mat.append(scores)
mutual_mat = np.array(mutual_mat)

CTZ
CTX


In [ ]:
mutual_mat_mean = mutual_mat.mean(axis=0)

In [ ]:
top_features = np.argsort(mutual_mat_mean)[::-1][:100000]
kmer_matrix_VT_top_features = kmer_matrix_VT1_fi[:,top_features]
selected_features_top = selected_features_fi[top_features]

In [ ]:
np.save('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_DNAmat_VT1_top_features.npy', kmer_matrix_VT_top_features) # save numpy array
np.save('/data/hoan/amromics/prediction/data/kmer_Fold'+aafold+'_DNAmat_VT1_features_top_features.npy', selected_features_top) # save numpy array

In [ ]:
stop

## Concat 3 datasets together (aafold for 0, 1, 2 and ...)

In [5]:
data1 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold1_DNAmat_VT1_top_features.npy') # save numpy array
feature1 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold1_DNAmat_VT1_features_top_features.npy') # save numpy array
df1 = pd.DataFrame(data = data1, columns = feature1)
#
data2 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold2_DNAmat_VT1_top_features.npy') # save numpy array
feature2 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold2_DNAmat_VT1_features_top_features.npy') # save numpy array
df2 = pd.DataFrame(data = data2, columns = feature2)
#
data3 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold3_DNAmat_VT1_top_features.npy') # save numpy array
feature3 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold3_DNAmat_VT1_features_top_features.npy') # save numpy array
df3 = pd.DataFrame(data = data3, columns = feature3)
#
data4 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold4_DNAmat_VT1_top_features.npy') # save numpy array
feature4 = np.load('/data/hoan/amromics/prediction/data/kmer_Fold4_DNAmat_VT1_features_top_features.npy') # save numpy array
df4 = pd.DataFrame(data = data4, columns = feature4)

In [6]:
df1.shape, df2.shape, df3.shape, df4.shape

((400, 100000), (400, 100000), (400, 100000), (453, 100000))

In [7]:
df_full = pd.concat([df1,df2,df3, df4], axis=0, ignore_index=True) 

In [8]:
df_full = df_full.fillna(0)

In [9]:
snp_mat = df_full.values

In [10]:
snp_mat.shape

(1653, 287260)

In [11]:
np.save('/data/hoan/amromics/prediction/data/kmer_full_mat_VT1_DNA.npy', snp_mat) # save numpy array